In [10]:
from moccasin import setup_notebook

setup_notebook()

In [11]:
from moccasin.config import get_active_netwrok
active_netwrok = get_active_netwrok
print(active_netwrok)

ImportError: cannot import name 'get_active_netwrok' from 'moccasin.config' (c:\programming\algo-trading\Algo-Trading\MoxProjectAlgoTrading\.venv\Lib\site-packages\moccasin\config.py)

In [12]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa 

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth):
    print(f"Starting balance of the weth: {weth.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)

    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print("After deposit:")
    print("ETH:", boa.env.get_balance(boa.env.eoa))
    print("WETH:", weth.balanceOf(boa.env.eoa))
    print("USDC:", usdc.balanceOf(boa.env.eoa))
    

def setup_script() -> Tuple[ABIContract, ABIContract,ABIContract,ABIContract]:
    print("Starting setup script...")

    active_netwrork = get_active_network()

    usdc = active_netwrork.manifest_named("usdc")
    weth = active_netwrork.manifest_named("weth")

    if active_netwrork.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc,weth)

def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...
Starting balance of the weth: 0
After deposit:
ETH: 999000000000000000000
WETH: 1000000000000000000
USDC: 100000000


In [14]:
active_netwrork = get_active_network()
usdc = active_netwrork.manifest_named("usdc")
weth = active_netwrork.manifest_named("weth")

In [15]:
from moccasin.config import get_config
config= get_config()
config.reload()
active_netwrork = config.get_active_network()
aavev3_pool_addresses_provider = active_netwrork.manifest_named("aavev3_pool_addresses_provider")
pool_address = aavev3_pool_addresses_provider.getPool()
print(pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [16]:
config.reload()
active_netwrork = config.get_active_network()
pool_contract = active_netwrork.manifest_named("pool", address = pool_address)

In [17]:
REFFERAL_CODE = 0
def deposit(pool_contract, token, amount):
    allowed_amout = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amout < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {token.name()} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFFERAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)
if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(totalCollateralBase,   
totalDebtBase,   
availableBorrowsBase,
currentLiquidationThreshold, 
ltv,    
healthFactor) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"User Account Data \n totalCollateralBase = {totalCollateralBase}, \n totalDebtBase = {totalDebtBase}, \n availableBorrowsBase = {availableBorrowsBase} \n currentLiquidationThreshold = {currentLiquidationThreshold} \n ltv = {ltv} \n healthFactor = {healthFactor}")



Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User Account Data 
 totalCollateralBase = 276163400000, 
 totalDebtBase = 0, 
 availableBorrowsBase = 221759210200 
 currentLiquidationThreshold = 8281 
 ltv = 8030 
 healthFactor = 115792089237316195423570985008687907853269984665640564039457584007913129639935


In [18]:
config.reload()
active_netwrork = config.get_active_network()
aave_protocol_data_provider = active_netwrork.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483f662091a1de47

In [19]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_netwrork.manifest_named('usdc', address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_netwrork.manifest_named('usdc', address = a_token[1])

print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [20]:
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)


100.0
1.0


In [21]:
a_usdc_balance_norm = a_usdc_balance / 1000000
a_weth_balance_norm = a_weth_balance/ 1000000000000000000
print(a_usdc_balance_norm )
print(a_weth_balance_norm )

100.0
1.0


In [32]:
config.reload()
active_netwrork = config.get_active_network()
eth_usd = active_netwrork.manifest_named("eth_usd")
print(eth_usd)
def get_price(feed_name:str) -> float:
    active_netwrork = config.get_active_network()
    price_feed = active_netwrork.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    print(f"Price from contracts is {price}")
    decimals = price_feed.decimals()
    print(f"Decimals from contracts is {decimals}")
    decimals_norm = 10 ** decimals
    return price / decimals_norm

usdc_price = get_price("usdc_usd")
print(usdc_price)

weth_price = get_price("eth_usd")
print(weth_price)

<eth_usd interface at 0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419>
Price from contracts is 99974000
Decimals from contracts is 8
0.99974
Price from contracts is 266166000000
Decimals from contracts is 8
2661.66


In [37]:
usdc_value = a_usdc_balance_norm * usdc_price 
weth_value = a_weth_balance_norm * weth_price
print(usdc_value)
print(weth_value)
overal_balance = usdc_value + weth_value
percent_weth = 100 / overal_balance * weth_value / 100
percent_usdc = 100 / overal_balance * usdc_value / 100
print(percent_weth)
print(percent_usdc)
target_usdc_value = 0.30
target_weth_value = 0.70
BUFFER = 0.1

need_rebalancing = {
    abs(percent_usdc - target_usdc_value) > BUFFER or abs(percent_weth - target_weth_value) > BUFFER
}

print(need_rebalancing)
print(pool_contract)

99.97399999999999
2661.66
0.9637989682919604
0.03620103170803951
{True}
<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>


In [42]:
#a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
#pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)



USDC : 0
WETH : 1000000000000000000
aUSDC : 100000000
aWETH : 0


In [ ]:
def print_token_balances():
    print(f"USDC : {usdc.balanceOf(boa.env.eoa)}")
    print(f"WETH : {weth.balanceOf(boa.env.eoa)}")
    print(f"aUSDC : {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"aWETH : {a_weth.balanceOf(boa.env.eoa)}")
    
    
print_token_balances()

USDC : 0
WETH : 1000000000000000000
aUSDC : 100000000
aWETH : 0



In [49]:

usdc_data = {"balance" : a_usdc_balance_norm, "price": usdc_price, "contract" : a_usdc}
weth_data = {"balance" : a_weth_balance_norm, "price": weth_price, "contract" : a_weth}
target_allocations = {"usdc": 0.3, "weth": 0.7}
def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }

trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)

weth_to_manage = trades["weth"]["trade"]
usdc_to_manage = trades["usdc"]["trade"]

print(weth_to_manage)
print(usdc_to_manage)


-0.27370746075757235
728.7056634725029


In [50]:
config.reload()
active_netwrork = config.get_active_network()
uniswap_swap_router = active_netwrork.manifest_named("uniswap_swap_router")

print(uniswap_swap_router)

<uniswap_swap_router interface at 0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45>


In [54]:
    # struct ExactInputSingleParams {
    #     address tokenIn;
    #     address tokenOut;
    #     uint24 fee;
    #     address recipient;
    #     uint256 deadline;
    #     uint256 amountIn;
    #     uint256 amountOutMinimum;
    #     uint160 sqrtPriceLimitX96;
    # }
amount_weth = abs(int(weth_to_manage * (10 ** weth.decimals())))
amount_usdc = usdc_to_manage * (10 ** usdc.decimals())
amount_usdc_min = int(usdc_to_manage * 0.9)
print(amount_weth)
print(amount_usdc)
weth.approve(uniswap_swap_router.address, amount_weth)
print("Let's swap!")
uniswap_swap_router.exactInputSingle(
    (
        weth.address,
        usdc.address,
        3000,
        boa.env.eoa,
        amount_weth,
        amount_usdc_min,
        0

    )
)

print_token_balances()

273707460757572352
728705663.472503
Let's swap!
USDC : 727768852
WETH : 726292539242427648
aUSDC : 100000000
aWETH : 0



In [55]:
deposit(pool_contract, usdc, usdc.balanceOf(boa.env.eoa))

print_token_balances()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_norm = a_usdc_balance / 1000000
a_weth_balance_norm = a_weth_balance / 1000000000000000000

usdc_val = a_usdc_balance_norm * usdc_price
weth_val = a_weth_balance_norm * weth_price

usdc_percent = usdc_val / (usdc_val + weth_val)
weth_percent = weth_val / (usdc_val + weth_val)

print(usdc_percent)
print(weth_percent)
                                        

Depositing USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
USDC : 0
WETH : 726292539242427648
aUSDC : 827768852
aWETH : 0

1.0
0.0


In [56]:
#deposit(pool_contract, weth, weth.balanceOf(boa.env.eoa))

print_token_balances()

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

a_usdc_balance_norm = a_usdc_balance / 1000000
a_weth_balance_norm = a_weth_balance / 1000000000000000000

usdc_val = a_usdc_balance_norm * usdc_price
weth_val = a_weth_balance_norm * weth_price

usdc_percent = usdc_val / (usdc_val + weth_val)
weth_percent = weth_val / (usdc_val + weth_val)

print(usdc_percent)
print(weth_percent)

Depositing Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
USDC : 0
WETH : 0
aUSDC : 827768852
aWETH : 726292539242427648

USDC : 0
WETH : 0
aUSDC : 827768852
aWETH : 726292539242427648

0.29976252467096126
0.7002374753290387


In [ ]:
tokens_arr = [trades["usdc"], trades["weth"]]
tokens_arr_sorted = sorted(tokens_arr, key=lambda x: x["trade"])
    
print(tokens_arr)
print(tokens_arr_sorted)

[{'contract': <usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>, 'trade': -0.27370746075757235}, {'contract': <usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>, 'trade': 728.7056634725029}]
[{'contract': <usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>, 'trade': -0.27370746075757235}, {'contract': <usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>, 'trade': 728.7056634725029}]
